In [ ]:
# https://www.epicurious.com/search/cucumbers - or whatever other search term

# Scrape 10 pages of cucumber search results, and save as a CSV file. Include the following fields:

# Tag/category
# Title
# Summary
# Rating (We'll only want the 2, not the 2 / 4)
# Would make again
# Link/URL
# Tip: You'll need to try/escape on some of these fields



# Loads of cucumbers:
### PART 1

In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [161]:

rows = []

for num in range(1, 11):
    url = f'https://www.epicurious.com/search/cucumbers?page={num}'
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    contents = doc.find_all(class_ = re.compile('.*-content-card'))
    print('Scraping', url)
    for content in contents:
        row = {}
        row['Tag'] = content.find(class_ = 'summary').strong.text
        row['Title'] = content.h4.text
        try:
            row['Summary'] = content.find(class_ = 'dek').text
        except:
            pass
        try:
            row['Rating'] = content.find(itemprop = 'ratingValue').text
        except:
            pass
        try:
            row['Make_Again_Pct'] = content.find(class_ = 'make-again-percentage').text
        except:
            pass
        row['Url'] = content.find(class_ = 'hed').a['href']
        rows.append(row)
    

Scraping https://www.epicurious.com/search/cucumbers?page=1
Scraping https://www.epicurious.com/search/cucumbers?page=2
Scraping https://www.epicurious.com/search/cucumbers?page=3
Scraping https://www.epicurious.com/search/cucumbers?page=4
Scraping https://www.epicurious.com/search/cucumbers?page=5
Scraping https://www.epicurious.com/search/cucumbers?page=6
Scraping https://www.epicurious.com/search/cucumbers?page=7
Scraping https://www.epicurious.com/search/cucumbers?page=8
Scraping https://www.epicurious.com/search/cucumbers?page=9
Scraping https://www.epicurious.com/search/cucumbers?page=10


In [162]:
df = pd.DataFrame(rows)
df = df[['Title', 'Tag', 'Summary', 'Url', 'Rating', 'Make_Again_Pct']]
df.head()

,Title,Tag,Summary,Url,Rating,Make_Again_Pct
0,Spicy Lightly Pickled Cucumbers,recipe,These quick pickles have just the right amount...,/recipes/food/views/spicy-lightly-pickled-cucu...,4,100%
1,Cucumbers with Ajo Blanco Sauce,recipe,Think of this as an all-purpose garlic sauce. ...,/recipes/food/views/cucumbers-with-ajo-blanco-...,4,100%
2,Cold Beef Tenderloin with Tomatoes and Cucumbers,recipe,Beef tenderloin is precious enough to baby on ...,/recipes/food/views/cold-beef-tenderloin-with-...,2,0%
3,Grain Salad with Tomatoes and Cucumbers,recipe,We call for semi-pearled grains because they c...,/recipes/food/views/farro-spelt-grain-salad-wi...,3,100%
4,Fried Fish Sandwiches with Cucumbers and Tarta...,recipe,Take your cast-iron pan to the grill for this ...,/recipes/food/views/fried-fish-sandwiches-with...,3,100%


In [49]:
df.to_csv('Epicurious_cucumbers.csv', index = False)

### PART 2

In [ ]:
# Then, open your search results csv, filter for ONLY recipes. Merge the following fields with your original recipes and save as a new CSV file:

# Ingredients
# Directions
# Tags
# Tip: If you use .find for your directions/ingredients, it'll print them all on one line. But if you use .find_all to separate them, it makes your life a lot harder! ...unless you just steal this code:



In [56]:
df = pd.read_csv('Epicurious_cucumbers.csv')

In [58]:
df = df[df.Tag == 'recipe']

In [166]:
def scrap_page(food):
    row = {}
    url = f"https://www.epicurious.com/{food['Url']}"
    print('Scraping', url)
    response = requests.get(url)
    contents = BeautifulSoup(response.text)
    row['Title'] = contents.find(class_ = 'title-source').h1.text.strip()
    row['Ingredients'] = list(map(lambda x: x.text, contents.find('ul', class_ = 'ingredients').find_all(class_ = 'ingredient')))
    row['Direction'] = list(map(lambda x: x.text.strip(), contents.find(class_ = 'preparation-group').find_all(class_ = 'preparation-step')))
    row['Tags'] = list(map(lambda x: x.text, contents.find(class_ = 'menus-tags content').find(class_ = 'tags').find_all('dt')))
    print(row)
    return pd.Series(row)

In [167]:
scraped_df = df.apply(scrap_page, axis = 1)

Scraping https://www.epicurious.com//recipes/food/views/spicy-lightly-pickled-cucumbers
{'Title': 'Spicy Lightly Pickled Cucumbers', 'Ingredients': ['2 lb. medium Persian cucumbers (about 12), cut lengthwise into spears', '1/4 cup white wine vinegar or unseasoned rice vinegar', '1 tsp. sugar', '3/4 tsp. crushed red pepper flakes', '2 tsp. kosher salt, plus more', '2 Tbsp. chopped dill', '2 Tbsp. fresh lemon juice'], 'Direction': ['Toss cucumbers in a large bowl with vinegar, sugar, red pepper flakes, and 2 tsp. salt. Chill, tossing once, at least 1 hour and up to 6 hours.', 'Just before serving, add dill and lemon juice and toss to combine. Taste and season with more salt if needed.'], 'Tags': ['Bon Appétit', 'Pickles', "Hors D'Oeuvre", 'Appetizer', 'Side', "New Year's Eve", 'Cocktail Party', 'Cucumber', 'Dill', 'Vinegar', 'Summer']}
Scraping https://www.epicurious.com//recipes/food/views/spicy-lightly-pickled-cucumbers
{'Title': 'Spicy Lightly Pickled Cucumbers', 'Ingredients': ['2 lb

{'Title': 'Cucumber-Seaweed Salad', 'Ingredients': ['3/4 ounce dried wakame seaweed, cut into 1-inch pieces', '2 cucumbers, peeled, seeded, and diced'], 'Direction': ['In a bowl, soak the wakame in warm water for 5 to 10 minutes. Drain in a colander. Combine the seaweed and cucumber in a bowl. In another bowl, whisk together the vinegar, coconut aminos, olive oil, lemon juice, ginger, and salt. Toss the vegetables with 2 tablespoons dressing. Taste and add more dressing if desired.'], 'Tags': ['HarperCollins', 'Salad', 'Side', 'Cucumber', 'Ginger', 'Quick & Easy', 'Quick and Healthy', 'Healthy', 'Dairy Free', 'Peanut Free', 'Wheat/Gluten-Free', 'Vegetarian', 'Vegan']}
Scraping https://www.epicurious.com//recipes/food/views/pineapple-shrimp-noodle-bowls
{'Title': 'Pineapple Shrimp Noodle Bowls', 'Ingredients': ['12 oz. pad Thai–style rice noodles', '1 lb. large shrimp', '12 oz. (1") fresh pineapple cubes (about 2 cups; from ½ pineapple)', '1 tsp. kosher salt', '2 Tbsp. soy sauce or tama

{'Title': 'Grilled Salmon with Meyer Lemons and Creamy Cucumber Salad', 'Ingredients': ['4 (6-ounce) skin-on wild-caught salmon fillets', '1/2 cup extra virgin olive oil, plus more for grilling', '1 tablespoon grated Meyer or regular lemon zest', 'Juice of 2 Meyer lemons (see head note) or regular lemons, plus 1 or 2 Meyer or regular lemons, sliced into 8 thin rounds', '1 tablespoon minced fresh dill', '1 tablespoon pink or regular sea salt'], 'Direction': ['To prepare the salmon: Brush the skin side of the salmon fillets with some olive oil. In a small bowl, combine the 1/2 cup olive oil, lemon zest, lemon juice, dill, and salt. Pour the mixture into a shallow dish and place the fillets skin side up in the dish. Cover and refrigerate for 1 hour.', 'Meanwhile, prepare the creamy cucumber salad: In a medium bowl, whisk together the mayonnaise, sour cream, lemon juice, dill, salt, and pepper. Fold in the cucumber until completely coated. Taste and adjust the lemon juice or salt. Cover an

AttributeError: ("'NoneType' object has no attribute 'h1'", 'occurred at index 15')

In [147]:
scraped_df.head()

,Title,Ingredients,Direction,Tags
0,Spicy Lightly Pickled Cucumbers,"[2 lb. medium Persian cucumbers (about 12), cu...","[Toss cucumbers in a large bowl with vinegar, ...","[Bon Appétit, Pickles, Hors D'Oeuvre, Appetize..."
1,Cucumbers with Ajo Blanco Sauce,[5 mini seedless or Persian cucumbers (about 1...,"[Preheat oven to 350°F. Cut cucumbers about 1""...","[Bon Appétit, Sauce, Condiment, Garlic, Almond..."
2,Cold Beef Tenderloin with Tomatoes and Cucumbers,"[1/4 cup extra-virgin olive oil, plus more, 1 ...",[Prepare a grill for 2-zone heat (for a charco...,"[Bon Appétit, Dinner, Beef Tenderloin, Beef, T..."
3,Grain Salad with Tomatoes and Cucumbers,"[2 cups semi-pearled farro or spelt, Kosher sa...",[Preheat oven to 350°F. Cook farro in a large ...,"[Bon Appétit, Salad, Side, Grains, Cucumber, T..."
4,Fried Fish Sandwiches with Cucumbers and Tarta...,"[1/2 cup mayonnaise, 1/4 cup finely grated lem...","[Mix mayonnaise, lemon zest and juice, relish,...","[Bon Appétit, Sandwich, Fish, Seafood, Fry, Be..."


In [163]:
merged_df = df.join(scraped_df, rsuffix = '_scraped').drop(columns = 'Title_scraped')
merged_df.head()

,Title,Tag,Summary,Url,Rating,Make_Again_Pct,Ingredients,Direction,Tags
0,Spicy Lightly Pickled Cucumbers,recipe,These quick pickles have just the right amount...,/recipes/food/views/spicy-lightly-pickled-cucu...,4,100%,"[2 lb. medium Persian cucumbers (about 12), cu...","[Toss cucumbers in a large bowl with vinegar, ...","[Bon Appétit, Pickles, Hors D'Oeuvre, Appetize..."
1,Cucumbers with Ajo Blanco Sauce,recipe,Think of this as an all-purpose garlic sauce. ...,/recipes/food/views/cucumbers-with-ajo-blanco-...,4,100%,[5 mini seedless or Persian cucumbers (about 1...,"[Preheat oven to 350°F. Cut cucumbers about 1""...","[Bon Appétit, Sauce, Condiment, Garlic, Almond..."
2,Cold Beef Tenderloin with Tomatoes and Cucumbers,recipe,Beef tenderloin is precious enough to baby on ...,/recipes/food/views/cold-beef-tenderloin-with-...,2,0%,"[1/4 cup extra-virgin olive oil, plus more, 1 ...",[Prepare a grill for 2-zone heat (for a charco...,"[Bon Appétit, Dinner, Beef Tenderloin, Beef, T..."
3,Grain Salad with Tomatoes and Cucumbers,recipe,We call for semi-pearled grains because they c...,/recipes/food/views/farro-spelt-grain-salad-wi...,3,100%,"[2 cups semi-pearled farro or spelt, Kosher sa...",[Preheat oven to 350°F. Cook farro in a large ...,"[Bon Appétit, Salad, Side, Grains, Cucumber, T..."
4,Fried Fish Sandwiches with Cucumbers and Tarta...,recipe,Take your cast-iron pan to the grill for this ...,/recipes/food/views/fried-fish-sandwiches-with...,3,100%,"[1/2 cup mayonnaise, 1/4 cup finely grated lem...","[Mix mayonnaise, lemon zest and juice, relish,...","[Bon Appétit, Sandwich, Fish, Seafood, Fry, Be..."


In [165]:
merged_df.to_csv('Epicurious_cucumbers.csv', index = False)

# I love waffles, so ...

In [35]:
rows = []

for num in range(1, 8):
    url = f'https://www.epicurious.com/search/waffles?page={num}'
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    contents = doc.find_all(class_ = re.compile('.*-content-card'))
    for content in contents:
        row = {}
        row['Tag'] = content.find(class_ = 'summary').strong.text
        row['Title'] = content.h4.text
        try:
            row['Summary'] = content.find(class_ = 'dek').text
        except:
            pass
        try:
            row['Rating'] = content.find(itemprop = 'ratingValue').text
        except:
            pass
        try:
            row['Make_Again_Pct'] = content.find(class_ = 'make-again-percentage').text
        except:
            pass
        row['Url'] = content.find(class_ = 'hed').a['href']
        rows.append(row)
    

Scraping https://www.epicurious.com/search/waffles?page=1
Scraping https://www.epicurious.com/search/waffles?page=2
Scraping https://www.epicurious.com/search/waffles?page=3
Scraping https://www.epicurious.com/search/waffles?page=4
Scraping https://www.epicurious.com/search/waffles?page=5
Scraping https://www.epicurious.com/search/waffles?page=6
Scraping https://www.epicurious.com/search/waffles?page=7


In [39]:
df = pd.DataFrame(rows)
df.head()

,Make_Again_Pct,Rating,Summary,Tag,Title,Url
0,100%,4,This classic recipe from the Gourmet archives ...,recipe,Buttermilk Waffles,/recipes/food/views/buttermilk-waffles
1,NaN,NaN,"Make Mama something special this Mother's Day,...",gallery,What to Cook This Weekend: May 10–12,/recipes-menus/what-to-cook-this-weekend-may-1...
2,NaN,NaN,But the Hong Kong egg waffle is more than just...,article,This Waffle Pan Will Increase Your Instagram F...,/expert-advice/hong-kong-egg-waffle-pan-article
3,0%,3,This pancake mix works just as well for sweet ...,recipe,Big-Batch Pancake and Waffle Mix,/recipes/food/views/big-batch-pancake-and-waff...
4,NaN,NaN,"The waffles, chicken wings, and killer egg san...",gallery,What to Cook This Weekend: June 14-16,/recipes-menus/what-to-cook-this-weekend-june-...


In [41]:
df = df[['Title', 'Tag', 'Summary', 'Url', 'Rating', 'Make_Again_Pct']]
df.head()

,Title,Tag,Summary,Url,Rating,Make_Again_Pct
0,Buttermilk Waffles,recipe,This classic recipe from the Gourmet archives ...,/recipes/food/views/buttermilk-waffles,4,100%
1,What to Cook This Weekend: May 10–12,gallery,"Make Mama something special this Mother's Day,...",/recipes-menus/what-to-cook-this-weekend-may-1...,NaN,NaN
2,This Waffle Pan Will Increase Your Instagram F...,article,But the Hong Kong egg waffle is more than just...,/expert-advice/hong-kong-egg-waffle-pan-article,NaN,NaN
3,Big-Batch Pancake and Waffle Mix,recipe,This pancake mix works just as well for sweet ...,/recipes/food/views/big-batch-pancake-and-waff...,3,0%
4,What to Cook This Weekend: June 14-16,gallery,"The waffles, chicken wings, and killer egg san...",/recipes-menus/what-to-cook-this-weekend-june-...,NaN,NaN


In [43]:
df.to_csv('Epicurious_waffles.csv', index = False)

In [154]:
waffle_df = pd.read_csv('Epicurious_waffles.csv')

In [155]:
waffle_df = waffle_df[waffle_df.Tag == 'recipe']

In [156]:
waffle_df.apply(scrap_page, axis = 1)

Scraping https://www.epicurious.com//recipes/food/views/buttermilk-waffles
Scraping https://www.epicurious.com//recipes/food/views/buttermilk-waffles
Scraping https://www.epicurious.com//recipes/food/views/big-batch-pancake-and-waffle-mix
Scraping https://www.epicurious.com//recipes/food/views/gluten-free-chocolate-and-buckwheat-waffles
Scraping https://www.epicurious.com//recipes/food/views/corn-waffles-with-strawberry-syrup
Scraping https://www.epicurious.com//recipes/food/views/crispy-cornmeal-bacon-waffles-109154
Scraping https://www.epicurious.com//recipes/food/views/coffee-flavored-belgian-waffles
Scraping https://www.epicurious.com//recipes/food/views/maple-butter
Scraping https://www.epicurious.com//recipes/food/views/passionfruit-curd
Scraping https://www.epicurious.com//recipes/food/views/overnight-waffles
Scraping https://www.epicurious.com//recipes/food/views/suzette-sauce
Scraping https://www.epicurious.com//recipes/food/views/elvis-peanut-butter-banana-waffles
Scraping ht

,Title,Ingredients,Direction,Tags
0,Buttermilk Waffles,"[3 cups all-purpose flour, 1 tablespoon baking...",[Preheat waffle iron. Put a rack in middle of ...,"[Breakfast, Brunch, Waffle, Kid-Friendly, Smal..."
3,Big-Batch Pancake and Waffle Mix,"[3/4 cup unsalted butter, cut into small piece...",[Place butter on a plate and freeze until very...,"[Sunday Stash, Breakfast, Brunch, Pancake, Waf..."
6,Gluten-Free Chocolate and Buckwheat Waffles,"[1/4 cup mixed buckwheat groats, black or whit...","[Preheat oven to 300°F. Toss buckwheat, 2 tsp....","[Bon Appétit, Breakfast, Waffle, Chocolate, Wh..."
14,Corn Waffles with Strawberry Syrup,"[3/4 cup agave nectar, preferably blue, 1 poun...",[Preheat oven to 200°F. Set a wire rack inside...,"[Bon Appétit, Waffle, Breakfast, Strawberry, C..."
23,Crispy Cornmeal-Bacon Waffles,[8 bacon slices],[Cook bacon in large skillet over medium heat ...,"[American, Pork, Breakfast, Brunch, Christmas,..."
35,Coffee-Flavored Belgian Waffles,"[1 1/2 cups all-purpose flour, 1/4 cup granula...","[Whisk flour, granulated sugar, brown sugar, b...","[Bon Appétit, Waffle, Breakfast, Coffee, Butte..."
44,Maple Butter,"[8 tablespoons (1 stick) salted butter, at roo...",[Combine the ingredients in a medium-size bowl...,"[Breakfast, Condiment, Condiment/Spread, Butte..."
47,Passionfruit Curd,"[10 passionfruit, halved and flesh scooped out...",[Sterilize the jars and lids you are using: wa...,"[HarperCollins, Dessert, Condiment/Spread, Con..."
52,Overnight Waffles,"[2 cups flour, 1 tablespoon sugar, 2 teaspoons...",[The night before you want to serve the waffle...,"[Waffle, Breakfast, Brunch, Pastry]"
62,Suzette Sauce,"[2/3 cup fresh orange juice, 2/3 cup sugar, 2 ...","[Bring orange juice, sugar, and Grand Marnier ...","[Bon Appétit, Sauce, Dessert, Breakfast, Frenc..."
